Dit notebook vernieuwd `stations.json`.

Hiervoor is een internetverbinding vereist (en dit script kan dus **niet** op MULAN gedraaid worden)

In [ ]:
import datetime
import json
import re
import urllib

In [ ]:
internet_access = False
try:
    r = urllib.request.urlopen("https://mesonet.agron.iastate.edu/geojson/networks.geojson")
    internet_access = True
    r.close()
except urllib.error.URLError as e:
    print("Could not connect to the Iowa State University - Environmetal Mesonet.")
    print("Please check if you are connected to the internet!")
    raise e from None

if internet_access:
    data = {'generated':datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%SZ'),'networks':{}, 'stations':{}}
    with urllib.request.urlopen("https://mesonet.agron.iastate.edu/geojson/networks.geojson") as response:
        networksjson = json.loads(response.read())
        for nw in networksjson['features']:
            if 'ASOS' in nw['id'] or 'AWOS' in nw['id']:
                abbr, name, polygon = nw['id'], nw['properties']['name'], nw['geometry']['coordinates']
                data['networks'][abbr] = {'abbr':abbr,'name':name,'polygon':polygon}
    networklist = list(data['networks'].keys())
    for i, network in enumerate(networklist):
        with urllib.request.urlopen(f"https://mesonet.agron.iastate.edu/geojson/network/{network}.geojson") as response:
            stationsjson = json.loads(response.read())
            for st in stationsjson['features']:
                abbr, name, elev, tzname = st['properties']['sid'], st['properties']['sname'], st['properties']['elevation'], st['properties']['tzname']
                lat, lon = st['geometry']['coordinates']
                data['stations'][abbr] = {'abbr':abbr,'name':name,'timezone':tzname,'latitude':lat,'longitude':lon,'elevation':elev}      
    with open('stations.json','w') as fh:
        json.dump(data,fh)